In [1]:
# mlrun: start-code

In [2]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [3]:
# mlrun: end-code

In [4]:
from mlrun import code_to_function, auto_mount

serving_fn = code_to_function(
    name='model-server-ensemble',
    project="royal-cyber",
    kind='serving',
    image='mlrun/mlrun'
).apply(auto_mount())

serving_fn.spec.default_class = 'ClassifierModel'
serving_fn.set_topology('router', 'mlrun.serving.VotingEnsemble')

serving_fn.add_model('lr', model_path="store://models/royal-cyber/my-training-job-tracking-main_LogisticRegression_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")
serving_fn.add_model('rf', model_path="store://models/royal-cyber/my-training-job-tracking-main_RandomForestClassifier_model#0:latest@32c8cd1816a14b5eb49d99ab92521b08")

In [5]:
function_address = serving_fn.deploy()

> 2021-08-20 23:11:17,945 [info] Starting remote function deploy
2021-08-20 23:11:18  (info) Deploying function
2021-08-20 23:11:18  (info) Building
2021-08-20 23:11:18  (info) Staging files and preparing base images
2021-08-20 23:11:18  (info) Building processor image
2021-08-20 23:11:19  (info) Build complete
2021-08-20 23:11:27  (info) Function deploy complete
> 2021-08-20 23:11:28,474 [info] function deployed, address=default-tenant.app.us-sales-eks.iguazio-cd0.com:32765


In [6]:
import json
import requests

X_test = [[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]
data = json.dumps({"inputs" : X_test})
data

'{"inputs": [[5.1, 3.5, 1.4, 0.2], [7.7, 3.8, 6.7, 2.2]]}'

In [7]:
for model in ["lr", "rf", "VotingEnsemble"]:
    # Inference via serving function
    print(serving_fn.invoke(f'/v2/models/{model}/infer', data))
    
    # Inference via HTTP post request
    print(requests.post(url=f"{function_address}/v2/models/{model}/infer", data=data).json())

{'id': '97d299c0-6d67-4cf3-8b0d-ec058f43b0ea', 'model_name': 'lr', 'outputs': [0, 2]}
{'id': '9c2cff8a-4bd2-4004-9eb6-f0cf6fec7d25', 'model_name': 'lr', 'outputs': [0, 2]}
{'id': 'cb9d4101-16b3-4dd4-bc7a-310bc39fe081', 'model_name': 'rf', 'outputs': [0, 2]}
{'id': '2c30b57f-4f1a-4186-b562-cfb97609580d', 'model_name': 'rf', 'outputs': [0, 2]}
{'id': '7d1c63ba-eb58-4799-a642-da8701faed54', 'model_name': [0, 2], 'outputs': [0, 2], 'model_version': 'v1'}
{'id': '5a7158c4-dae8-4681-967a-183ea75b6823', 'model_name': [0, 2], 'outputs': [0, 2], 'model_version': 'v1'}
